### Data cleaning

In [2]:
import pandas as pd
import numpy as np

In [3]:
yanki_df = pd.read_csv(r'dataset\raw_data\yanki_ecommerce.csv')

In [ ]:
yanki_df.info()


In [4]:
yanki_df.dropna(subset= ['Customer_ID', 'Order_ID', 'State'], inplace=True)

In [ ]:
yanki_df.info()

In [5]:
yanki_df['Order_Date'] = pd.to_datetime(yanki_df['Order_Date'], dayfirst=True)


In [ ]:
yanki_df.info()

In [ ]:
customer_df = yanki_df[['Customer_ID', 'Customer_Name', 'Phone_Number', 'Email']].copy().drop_duplicates().reset_index(drop=True)
customer_df.tail()

In [ ]:
product_df = yanki_df[['Product_ID', 'Product_Name', 'Category', 'Brand', 'Price']].copy().drop_duplicates().reset_index(drop=True)
product_df.head()

In [ ]:
shipping_df = yanki_df[['Customer_ID', 'Country', 'State', 'City', 'Shipping_Address', 'Postal_Code']].copy().drop_duplicates().reset_index(drop=True)
shipping_df.index.name = 'Shipping_ID'
shipping_df = shipping_df.reset_index()
shipping_df.head(4)

In [ ]:
Order_df = yanki_df[['Order_ID', 'Customer_ID', 'Product_ID', 'Order_Date', 'Total_Price', 'Quantity']].copy().drop_duplicates().reset_index(drop=True)
Order_df.tail(5)

In [ ]:
payment_df = yanki_df[[ 'Order_ID', 'Payment_Method', 'Transaction_Status']].copy().drop_duplicates().reset_index(drop=True)
payment_df.head(5)

In [9]:
yanki_df.columns

Index(['Order_ID', 'Customer_ID', 'Customer_Name', 'Product_ID',
       'Product_Name', 'Brand', 'Category', 'Price', 'Quantity', 'Total_Price',
       'Order_Date', 'Shipping_Address', 'City', 'State', 'Country',
       'Postal_Code', 'Email', 'Phone_Number', 'Payment_Method',
       'Transaction_Status'],
      dtype='object')

In [14]:
customer_df.to_csv(r'dataset/clean_data/Customers.csv', index=False)
shipping_df.to_csv(r'dataset/clean_data/Shipping.csv', index=False)
Order_df.to_csv(r'dataset/clean_data/Order.csv', index=False)
payment_df.to_csv(r'dataset/clean_data/Payment.csv', index=False)
product_df.to_csv(r'dataset/clean_data/Product.csv', index=False)

### Data Loading

In [ ]:
!pip install psycopg2

In [10]:
import psycopg2

In [11]:
def get_db_connection ():
    connection = psycopg2.connect(
       host = 'localhost',
       database = 'yanki_ecommerce',
       user = 'postgres',
       password = 'Cisco123'
    )
    return connection

In [18]:
conn = get_db_connection()

In [ ]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = ''' 
                            CREATE SCHEMA IF NOT EXISTS yanki;
                            
                            DROP TABLE IF EXISTS yanki.customers CASCADE;
                            DROP TABLE IF EXISTS yanki.products CASCADE;
                            DROP TABLE IF EXISTS yanki.shipping CASCADE;
                            DROP TABLE IF EXISTS yanki.orders CASCADE;
                            DROP TABLE IF EXISTS yanki.payment CASCADE;
                            
                            CREATE TABLE IF NOT EXISTS yanki.customers (
                                Customer_ID UUID PRIMARY KEY,
                                Customer_Name TEXT,
                                Phone_NUmber TEXT,
                                Email TEXT
                            );

                            CREATE TABLE IF NOT EXISTS yanki.products (
                                Product_ID UUID PRIMARY KEY,
                                Product_Name TEXT,
                                Category TEXT,
                                Brand TEXT,
                                Price FLOAT
                            );
                            
                            CREATE TABLE IF NOT EXISTS yanki.shipping (
                                Shipping_ID INTEGER PRIMARY KEY,
                                Customer_ID UUID,
                                Country TEXT,
                                State TEXT,
                                City TEXT,
                                Shipping_Address TEXT,
                                Postal_Code TEXT,
                                FOREIGN KEY (Customer_ID) references yanki.customers(Customer_ID)
                            );
                            
                            CREATE TABLE IF NOT EXISTS yanki.orders (
                                Order_ID UUID PRIMARY KEY,
                                Customer_ID UUID,
                                Product_ID UUID,
                                Order_Date DATE,
                                Total_Price DECIMAL,
                                Quantity INTEGER,
                                FOREIGN KEY(Product_ID) references yanki.products(Product_ID), 
                                FOREIGN KEY(Customer_ID) references yanki.customers(Customer_ID) 

                            );
                            
                            CREATE TABLE IF NOT EXISTS yanki.payment (
                                Order_ID UUID,
                                Payment_Method TEXT,
                                Transaction_Status TEXT,
                                FOREIGN KEY(Order_ID) references yanki.orders(Order_ID)
                            );'''
                            
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()


In [91]:
create_tables()

In [92]:
import csv
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' 
                           INSERT INTO yanki.customers (Customer_ID, Customer_Name, Phone_NUmber, Email)
                           VALUES (%s, %s, %s, %s);''',
                           row  
                        ) 
    conn.commit()
    cursor.close()
    conn.close()
    
csv_file_path = r'dataset\clean_data\Customers.csv'
load_data_from_csv(csv_file_path)
    

In [93]:
import csv
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' 
                           INSERT INTO yanki.products (Product_ID, Product_Name, Category, Brand, Price)
                           VALUES (%s, %s, %s, %s, %s);''',
                           row  
                        ) 
    conn.commit()
    cursor.close()
    conn.close()
    
csv_file_path = r'dataset\clean_data\Product.csv'
load_data_from_csv(csv_file_path)
    

In [94]:
import csv
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' 
                           INSERT INTO yanki.shipping (Shipping_ID, Customer_ID, Country, State, City, Shipping_Address, Postal_Code)
                           VALUES (%s, %s, %s, %s, %s, %s, %s);''',
                           row  
                        ) 
    conn.commit()
    cursor.close()
    conn.close()
    
csv_file_path = r'dataset\clean_data\Shipping.csv'
load_data_from_csv(csv_file_path)
    

In [95]:
import csv
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' 
                           INSERT INTO yanki.orders(Order_ID, Customer_ID, Product_ID, Order_Date, Total_Price, Quantity)
                           VALUES (%s, %s, %s, %s, %s, %s);''',
                           row  
                        ) 
    conn.commit()
    cursor.close()
    conn.close()
    
csv_file_path = r'dataset\clean_data\Order.csv'
load_data_from_csv(csv_file_path)
    

In [97]:
import csv
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' 
                           INSERT INTO yanki.payment (Order_ID, Payment_Method, Transaction_Status)
                           VALUES (%s, %s, %s);''',
                           row  
                        ) 
    conn.commit()
    cursor.close()
    conn.close()
    
csv_file_path = r'dataset\clean_data\Payment.csv'
load_data_from_csv(csv_file_path)
    

UniqueViolation: duplicate key value violates unique constraint "payment_pkey"
DETAIL:  Key (order_id)=(e32af09a-b6ab-497e-af3a-331e4d4ed6e7) already exists.


Index(['Product_ID', 'Product_Name', 'Category', 'Brand', 'Price'], dtype='object')